In [2]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices("GPU")
print(gpus)
for g in gpus:
    tf.config.experimental.set_memoty_growth(g, True)

Num GPUs Available:  0
[]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TerminateOnNaN, ModelCheckpoint
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
import numpy as np

# Load CSV data
train_data = pd.read_csv(r"D:\Projects\Digit-Recognizer-ML_Flask\train.csv")

# Prepare CSV dataset
x_csv = train_data.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0
y_csv = to_categorical(train_data['label'].values)

# Load MNIST dataset
(x_mnist_train, y_mnist_train), (x_mnist_test, y_mnist_test) = mnist.load_data()
x_mnist_train = x_mnist_train.reshape(-1, 28, 28, 1) / 255.0
y_mnist_train = to_categorical(y_mnist_train, 10)

x_mnist_test = x_mnist_test.reshape(-1, 28, 28, 1) / 255.0
y_mnist_test = to_categorical(y_mnist_test, 10)

# Concatenate CSV and MNIST datasets
x_combined = np.concatenate([x_csv, x_mnist_train, x_mnist_test], axis=0)
y_combined = np.concatenate([y_csv, y_mnist_train, y_mnist_test], axis=0)

# Train-validation split for combined dataset
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(x_combined, y_combined, test_size=0.1, random_state=random_seed)

# Define the model
model = Sequential([
        ZeroPadding2D(padding=(1, 1), input_shape=(28, 28, 1)),
        Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same',
                        activation ='relu', input_shape = (28,28,1)),
        BatchNormalization(),
        Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',
                        activation ='relu'),
        BatchNormalization(),
        Activation(activations.relu),
        MaxPooling2D(pool_size=(2,2)),
        ZeroPadding2D(padding=(1, 1)),
        Dropout(0.2),

        Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',
                        activation ='relu'),
        BatchNormalization(),
        Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',
                        activation ='relu'),
        BatchNormalization(),
        Activation(activations.relu),
        MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        Dropout(0.2),

        Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same',
                        activation ='relu'),
        BatchNormalization(),
        Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same',
                        activation ='relu'),
        BatchNormalization(),
        Activation(activations.relu),
        MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        Dropout(0.3),

        Flatten(),
        Dense(128, activation = "relu"),
        Dropout(0.4),
        Dense(10, activation = "softmax")
])

checkpoint = ModelCheckpoint(r'D:\Projects\Digit-Recognizer\mnist-cnn.keras', # your path here
                             monitor='val_accuracy',     # Accuracy on the validation set
                             save_best_only=True,        # Save only the best model
                             verbose=1)   
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience=3, 
    verbose=1, 
    factor=0.5, 
    min_lr=0.00001
)
early_stopping = EarlyStopping(
    monitor='val_loss',        # Моніторинг валідаційних втрат
    patience=5,                # Чекати 5 епох без покращення перед зупинкою
    restore_best_weights=True,  # Відновити ваги найкращої моделі
    mode='auto'
)
terminate_nan = TerminateOnNaN()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image Data Generator for augmentation
datagen = ImageDataGenerator(
   rotation_range=10,          # Randomly rotate images by up to 10 degrees
   zoom_range=0.1,             # Randomly zoom images by up to 10%
   width_shift_range=0.1,      # Randomly shift images horizontally by up to 10%
   height_shift_range=0.1,     # Randomly shift images vertically by up to 10%
)

# Train the model
batch_size = 32
history = model.fit(datagen.flow(x_combined, y_combined, batch_size=batch_size),
                    epochs=20,
                    validation_data=(X_val, Y_val),
                    steps_per_epoch=x_combined.shape[0] // batch_size,
                    verbose=1,
                    callbacks=[learning_rate_reduction, early_stopping, terminate_nan, checkpoint],
                    shuffle=True)

# Evaluate the model
model.evaluate(X_val, Y_val)

# Plot accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

def imshow():
    n = 10
    plt.figure(figsize=(10,6))
    for images, labels in mnist.take(1):
        for i in range(n):
            img = images[i]
            img_tensor = tf.expand_dims(img, axis=0)
            pred = model.nn(img_tensor)
            pred = tf.squeeze(pred, axis=0)
            pred = pred.numpy()
            ax = plt.subplot(3, n, i + 1 + n)
            plt.imshow(img, cmap="gist_gray")
            ma = pred.max()
            res = np.where(pred == ma)
            plt.title(res[0][0])
            plt.axis('off')
            ax.get_yaxis().set_visible(False)
    plt.show()
imshow()

AttributeError: module 'keras.api.datasets.mnist' has no attribute 'take'

<Figure size 1000x600 with 0 Axes>

In [ ]:
model.summary()